## Подбор функций акивации для LSTM моделей

### Подключение библиотек

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import LSTM, Dropout
from tensorflow.keras.optimizers import Adam

import sys
import os
sys.path.append(os.path.abspath('..'))
from helpful_functions import train_val_test_split, \
                              create_sequences, \
                              train_and_evaluate_model

### Подготовка данных

In [2]:
file_path = '../dataset/preprocessed_data.csv'
df = pd.read_csv(file_path)

df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)
df.sort_index(inplace=True)

target_feature = 'Global_active_power'
test_ratio = 0.10
val_ratio = 0.10

train_data, val_data, test_data = train_val_test_split(df, val_ratio, test_ratio)

scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train_data)
val_scaled = scaler.transform(val_data)
test_scaled = scaler.transform(test_data)

target_idx = df.columns.get_loc(target_feature)

features_count = train_data.shape[1]

### Обучение моделей

In [3]:
lr = 0.01
window_size = 30

X_train, y_train = create_sequences(train_scaled, target_idx, window_size)
X_val, y_val = create_sequences(val_scaled, target_idx, window_size)
X_test, y_test = create_sequences(test_scaled, target_idx, window_size)

results = {}

activations_to_test = {
    ("tanh", "tanh"),
    ("relu", "tanh"),
    ("sigmoid", "tanh"),
    ("tanh", "relu"),
    ("relu", "relu"),
    ("sigmoid", "relu"),
    ("tanh", "sigmoid"),
    ("relu", "sigmoid"),
    ("sigmoid", "sigmoid")
}

results = {}

for idx, (act1, act2) in enumerate(activations_to_test):
    print(f"\n{'=' * 50}")
    print(f"Тестирование комбинации {idx + 1}/{len(activations_to_test)}: "
          f"LSTM1={act1}, LSTM2={act2}")
    model_name = f"LSTM_{act1}_{act2}"

    model = Sequential()
    model.add(Input(shape=(window_size, features_count)))
    model.add(LSTM(100, activation=act1, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(50, activation=act2))
    model.add(Dense(1))
    
    result = train_and_evaluate_model(
        model, model_name, X_train, y_train, X_val, y_val, X_test, y_test,
        lr, window_size, scaler, target_feature, df.columns
    )
    
    results[model_name] = result


Тестирование комбинации 1/9: LSTM1=tanh, LSTM2=relu
LSTM_tanh_relu | val_loss: 0.0003 | MSE_val: 0.0409

Тестирование комбинации 2/9: LSTM1=relu, LSTM2=tanh
LSTM_relu_tanh | val_loss: 0.0006 | MSE_val: 0.0754

Тестирование комбинации 3/9: LSTM1=sigmoid, LSTM2=relu
LSTM_sigmoid_relu | val_loss: 0.0004 | MSE_val: 0.0510

Тестирование комбинации 4/9: LSTM1=relu, LSTM2=sigmoid
LSTM_relu_sigmoid | val_loss: 0.0059 | MSE_val: 0.7256

Тестирование комбинации 5/9: LSTM1=tanh, LSTM2=tanh
LSTM_tanh_tanh | val_loss: 0.0003 | MSE_val: 0.0398

Тестирование комбинации 6/9: LSTM1=sigmoid, LSTM2=tanh
LSTM_sigmoid_tanh | val_loss: 0.0005 | MSE_val: 0.0614

Тестирование комбинации 7/9: LSTM1=tanh, LSTM2=sigmoid
LSTM_tanh_sigmoid | val_loss: 0.0003 | MSE_val: 0.0417

Тестирование комбинации 8/9: LSTM1=sigmoid, LSTM2=sigmoid
LSTM_sigmoid_sigmoid | val_loss: 0.0004 | MSE_val: 0.0455

Тестирование комбинации 9/9: LSTM1=relu, LSTM2=relu
LSTM_relu_relu | val_loss: 0.0013 | MSE_val: 0.1622


### Вывод информации всех результатов обучения модели

In [4]:
results_df = pd.DataFrame(results.values())
print(results_df.sort_values(by=['mse_val_dn']))

             model_type  learning_rate  window_size  train_loss  val_loss  \
4        LSTM_tanh_tanh           0.01           30    0.000457  0.000326   
0        LSTM_tanh_relu           0.01           30    0.000458  0.000336   
6     LSTM_tanh_sigmoid           0.01           30    0.000465  0.000342   
7  LSTM_sigmoid_sigmoid           0.01           30    0.000504  0.000373   
2     LSTM_sigmoid_relu           0.01           30    0.000540  0.000418   
5     LSTM_sigmoid_tanh           0.01           30    0.000497  0.000504   
1        LSTM_relu_tanh           0.01           30    0.000495  0.000618   
8        LSTM_relu_relu           0.01           30    0.000534  0.001330   
3     LSTM_relu_sigmoid           0.01           30    0.009387  0.005947   

   mse_val_dn  mse_test_dn  
4    0.039811     0.033824  
0    0.040938     0.035352  
6    0.041734     0.035343  
7    0.045474     0.040482  
2    0.050999     0.044883  
5    0.061447     0.056551  
1    0.075381     0.061415

### Вывод информации лучшей модели

In [5]:
lstm_results = results_df[results_df['model_type'].str.startswith('LSTM_')]
best_lstm = lstm_results.nsmallest(1, 'mse_val_dn').iloc[0]

print("\nЛучшая LSTM (по валидации):")
print(best_lstm[['model_type', 'learning_rate', 'window_size', 'mse_val_dn', 'mse_test_dn']])


Лучшая LSTM (по валидации):
model_type       LSTM_tanh_tanh
learning_rate              0.01
window_size                  30
mse_val_dn             0.039811
mse_test_dn            0.033824
Name: 4, dtype: object
